Importing libraries

In [ ]:
import tensorflow as tf
from keras import backend
from keras.dtensor import layout_map as layout_map_lib

import time
import mouse
import keyboard
from importlib.resources import is_resource
from pynput.keyboard import Key, Controller
from pynput.mouse import Button, Controller

import cv2
import numpy as np
import PIL.ImageGrab


import keras
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation

Collecting training data (during the game)

In [ ]:
def screenshot():
    screenshot = PIL.ImageGrab.grab()

    img = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2GRAY)
    img = cv2.resize(img, (img.shape[1] // 2, img.shape[0] // 2))

    img = cv2.Canny(img, 40, 40)

    kernel = np.ones((1, 1), np.uint8) #uinSt8 - натуральные толщина слоя
    img = cv2.dilate(img, kernel, iterations=5)
    img = cv2.erode(img, kernel, iterations=5)
    
    return img

In [66]:
cnt_up = 0
cnt_down = 0
cnt_left = 0 
cnt_right = 0
cnt_none = 0

In [151]:
while True:
    try:
        if mouse.is_pressed('left'):
            break
    except:
        break

time.sleep(1)
    

while True:
    try:
        if mouse.is_pressed('left'):
            break
    except:
        break
    
time.sleep(1)
    
    
while True:
    try:
        if mouse.is_pressed('left'):
            break
    except:
        break
    
    
while True:
    try:
        if keyboard.is_pressed('enter'):
            break
        
        elif keyboard.is_pressed('up'):
            screen = screenshot()
            cv2.imwrite("Moves/Up/" + str(cnt_up) + ".jpg", screen)
            cnt_up += 1
            time.sleep(0.1)
        
        elif keyboard.is_pressed('down'):
            screen = screenshot()
            cv2.imwrite("Moves/Down/" + str(cnt_down) + ".jpg", screen)
            cnt_down += 1
            time.sleep(0.1)
        
        elif keyboard.is_pressed('left'):
            screen = screenshot()
            cv2.imwrite("Moves/Left/" + str(cnt_left) + ".jpg", screen)
            cnt_left += 1
            time.sleep(0.1)
        
        elif keyboard.is_pressed('right'):
            screen = screenshot()
            cv2.imwrite("Moves/Right/" + str(cnt_right) + ".jpg", screen)
            
            cnt_right += 1
            time.sleep(0.1)
            
        elif keyboard.is_pressed('enter'):
            break

        else:
            screen = screenshot()
            cv2.imwrite("Moves/None/" + str(cnt_none) + ".jpg", screen)
            cnt_none += 1
            time.sleep(0.1)
            
    except:
        break
    

Сonverting images into arrays and compiling a training dataset

In [ ]:
down = np.zeros(shape=(1000, 720, 1280))

for i in range(281):
    img = cv2.imread('Train/Down/Down(' + str(i) + ').jpg')
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
    img = np.array(img)
    down[i] = img

In [ ]:
left = np.zeros(shape=(1000, 720, 1280))

for i in range(606):
    img = cv2.imread('Train/Left/Left(' + str(i) + ').jpg')
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
    img = np.array(img)
    left[i] = img

In [ ]:
right = np.zeros(shape=(1000, 720, 1280))

for i in range(606):
    img = cv2.imread('Train/Right/Right(' + str(i) + ').jpg')
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
    img = np.array(img)
    right[i] = img

In [ ]:
up = np.zeros(shape=(1000, 720, 1280))

for i in range(208):
    img = cv2.imread('Train/Up/Up(' + str(i) + ').jpg')
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
    img = np.array(img)
    up[i] = img

In [ ]:
none = np.zeros(shape=(1000, 720, 1280))

for i in range(208):
    img = cv2.imread('Train/None/None(' + str(i) + ').jpg')
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
    img = np.array(img)
    up[i] = img

In [ ]:
for num in range(1000):
    x_train = np.zeros(shape=(500, 720, 1280), dtype=float)
    y_train = np.zeros(shape=(500), dtype=float)
    
    for i in range(500):
        x = np.random.randint(0,5) # всего 5 вариантов
        y_train[i] = x

        if x == 0:
            j = np.random.randint(0, 208)
            img = cv2.imread('Moves/Up/Up(' + str(j) + ').jpg')
        
        elif x == 1:
            j = np.random.randint(0, 281)
            img = cv2.imread('Moves/Down/Down(' + str(j) + ').jpg')
            
        elif x == 2:
            j = np.random.randint(0, 606)
            img = cv2.imread('Moves/Left/Left(' + str(j) + ').jpg')
            
        elif x == 3:
            j = np.random.randint(0, 606)
            img = cv2.imread('Moves/Right/Right(' + str(j) + ').jpg')

        elif x == 4:
            j = np.random.randint(0, 10189)
            img = cv2.imread('Moves/None/None(' + str(j) + ').jpg')
            
        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
        img = np.array(img)
        
        x_train[i] = img
        
    np.save('Train_data/x_train_' + str(num) + '.npy', x_train)
    np.save('Train_data/y_train_' + str(num) + '.npy', y_train)

Neural network architecture

In [1]:
import keras 
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D

In [3]:
model = keras.Sequential([
    Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(720, 1280, 1)), #число фильров, размер ядер фильров, размер выходной карты признаков такая же, ...
    MaxPooling2D((2, 2), strides=2), #размер окна и шаг сканирования (в данном случае не пересекаются)
    
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Dropout(0.2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(5,  activation='sigmoid')
]) 

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(720, 1280, 1), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(5))
model.add(Activation('softmax'))


In [ ]:
model = keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation="tanh", input_shape=(720, 1280)),
    tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(16, (3, 3), activation="tanh"),
    tf.keras.layers.AveragePooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(5, activation="sigmoid")
    ])

Start training

In [4]:
model.compile(optimizer='nadam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
for i in range(0, 100):
    x_train = np.load('Train_data/x_train_' + str(i) + '.npy')
    y_train = np.load('Train_data/y_train_' + str(i) + '.npy')
    y_train_cat = keras.utils.to_categorical(y_train, 5)
    
    model.fit(x_train, y_train_cat, batch_size=10, epochs=5, validation_split=0.2)
    print()


model.save('model.h5')

Checking accuracy on the whole set and on a separate image

In [ ]:
model.evaluate(x_train, y_train_cat)

In [ ]:
x = np.expand_dims(x_train[200], axis = 0)   
res = model.predict(x)
np.argmax(res)

Start the game and check the performance

In [3]:
model = keras.models.load_model('model.h5')

In [ ]:
time.sleep(5)
kb = Controller()
mouse.click('left')

while True:
    try:
        screen = screenshot()
        x = np.expand_dims(screen, axis = 0)   
        res = model.predict(x)
        move = np.argmax(res)
        
        if move == 0:
            kb.press(Key.up)
        
        elif move == 1:
            kb.press(Key.down)
            
        elif move == 2:
            kb.press(Key.left)
            
        elif move == 3:
            kb.press(Key.right)
            
        time.sleep(0.1)
        
    except:
        break